###import packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import StratifiedShuffleSplit, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix


###import train and test data and check out the sizes of these two files

In [2]:
train=pd.read_csv('train.csv')
train.shape

(2013, 5)

In [3]:
test=pd.read_csv('test.csv')
test.shape

(4700, 3)

###import the bid file (which is big), and sort it by auction and time, and calculate the time intervel of each bid from the last bid. If it is the first bid in an auction, use the averaged time interval of all bids: 52858888310.29  save the time interval as a new column "speed"

In [ ]:
bids=pd.read_csv('bids.csv')
sorted=bids.sort(['auction','time'])

sorted.to_csv('sorted.csv')

f=open('sorted.csv','r')
h=open('sorted_speed.csv','w')
header=f.readline().strip()
h.write(header+",speed"+"\n")
num_lines =sum(1 for line in open('sorted.csv'))
pre_auction='00270'
pre_time=9699049894736842
for i in range(0,num_lines-1):
    
    data=f.readline().strip()
    auction=data.split(",")[3]
    time=float(data.split(",")[6])
    if auction==pre_auction:
        speed=time-pre_time
        pre_time=time
    else:
        speed=0
        pre_auction=auction
        pre_time=time
    if speed==0:
        speed=52858888310.29
    h.write(data + "," + str(speed) + "\n")
h.close()

In [2]:
sorted_speed=pd.read_csv('sorted_speed2.csv')
sorted_speed.shape

(7656334, 11)

In [4]:
sorted_speed.head()

,Unnamed: 0,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url,speed
0,5826422,5826422,92af1e40713e077ef87f5352fb56772fjnzm7,00270,mobile,phone563,9699049894736842,us,78.188.245.105,vuaaeraf3dqz299,5.285889e+10
1,5834284,5834284,a939f51234ad2c44eb9ccc84e754f41foiv7g,00270,mobile,phone63,9699098210526315,us,12.142.135.122,fbpq1mgtpgnkqnk,4.831579e+10
2,5850448,5850448,92af1e40713e077ef87f5352fb56772fjnzm7,00270,mobile,phone735,9699195789473684,us,2.86.254.154,nn2g552xadgkfch,9.757895e+10
3,5884054,5884054,9655ccc7c0c193f1549475f02c54dce45kjw7,00270,sporting goods,phone573,9699392473684210,us,149.152.163.145,vasstdc27m7nks3,1.966842e+11
4,5898318,5898318,9655ccc7c0c193f1549475f02c54dce45kjw7,00270,sporting goods,phone2294,9699474473684210,us,219.254.45.139,vasstdc27m7nks3,8.200000e+10


## here I'm trying to find within each auction, the consecutive bidding from the same bidder. 

In [25]:
a=sorted_speed.loc[sorted_speed.shift(1)['bidder_id'] != sorted_speed['bidder_id']]

In [26]:
a['ind']=a.index

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [27]:
a['consec_bid']=-a['ind']+a.shift(-1)['ind']

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [28]:
a.head()

,Unnamed: 0,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url,speed,last_time,start_time,stop_time,bid_time_percent,ind,consec_bid
0,5826422,5826422,92af1e40713e077ef87f5352fb56772fjnzm7,00270,mobile,phone563,9.699050e+15,us,78.188.245.105,vuaaeraf3dqz299,5.285889e+10,1.016300e+13,9.699050e+15,9.709210e+15,-0.000010,0,1
1,5834284,5834284,a939f51234ad2c44eb9ccc84e754f41foiv7g,00270,mobile,phone63,9.699098e+15,us,12.142.135.122,fbpq1mgtpgnkqnk,4.831579e+10,1.016300e+13,9.699050e+15,9.709210e+15,0.004744,1,1
2,5850448,5850448,92af1e40713e077ef87f5352fb56772fjnzm7,00270,mobile,phone735,9.699196e+15,us,2.86.254.154,nn2g552xadgkfch,9.757895e+10,1.016300e+13,9.699050e+15,9.709210e+15,0.014345,2,1
3,5884054,5884054,9655ccc7c0c193f1549475f02c54dce45kjw7,00270,sporting goods,phone573,9.699392e+15,us,149.152.163.145,vasstdc27m7nks3,1.966842e+11,1.016300e+13,9.699050e+15,9.709210e+15,0.033698,3,3
6,5911709,5911709,84c4b58a1491da3a30710bcdf61f87054xfq6,00270,mobile,phone2,9.699551e+15,us,247.124.231.180,qi80ms9ikpku6g3,7.736842e+09,1.016300e+13,9.699050e+15,9.709210e+15,0.049265,6,1


## here I find the maximun consecutive bidding within one auction of each biddre

In [29]:
a.groupby(['auction','bidder_id'])['consec_bid'].max().groupby(level=1).mean().head()

bidder_id
001068c415025a009fee375a12cff4fcnht8y    1
002d229ffb247009810828f648afc2ef593rb    2
0030a2dd87ad2733e0873062e4f83954mkj86    1
003180b29c6a5f8f1d84a6b7b6f7be57tjj1o    1
00486a11dff552c4bd7696265724ff81yeo9v    1
Name: consec_bid, dtype: float64

### here I'm calculating how long each auction last by substracting the end time from the start time. Also I save the start time and end time of each auction into new columns

In [71]:
last_time=sorted_speed.groupby(['auction'])['time'].last()-sorted_speed.groupby(['auction'])['time'].first()
start_time=sorted_speed.groupby(['auction'])['time'].first()
stop_time=sorted_speed.groupby(['auction'])['time'].last()

In [29]:
start_time

auction
00270      9.699050e+15
008vv      9.759369e+15
00cwr      9.695642e+15
00do0      9.759324e+15
00hjy      9.759410e+15
00hm5      9.631996e+15
00llo      9.761695e+15
00sj9      9.635671e+15
010lx      9.637006e+15
012sd      9.759251e+15
012wh      9.759244e+15
0143v      9.632040e+15
014m6      9.759324e+15
01b3e      9.759266e+15
01dgm      9.639211e+15
...
zz04r      9.760212e+15
zz1cb      9.759309e+15
zz2r1      9.636429e+15
zz2zx      9.631919e+15
zz4dz      9.759296e+15
zz6y6      9.695612e+15
zzal8      9.761711e+15
zzdqu      9.695938e+15
zzeld      9.632026e+15
zzjoj      9.638380e+15
zznnc      9.759596e+15
zzoy7      9.631994e+15
zztha      9.767389e+15
zzzlx      9.759265e+15
zzzy7      9.641482e+15
Name: time, Length: 13963, dtype: float64

In [16]:
last_time.to_csv('last_time.csv')

In [30]:
start_time.to_csv('start_time.csv')

In [72]:
stop_time.to_csv('stop_time.csv')

In [11]:
last_time=pd.read_csv('last_time.csv')

In [4]:
start_time=pd.read_csv('start_time.csv')
stop_time=pd.read_csv('stop_time.csv')

In [12]:
sorted_speed=pd.merge(sorted_speed,last_time,how='outer',on='auction')

In [6]:
sorted_speed=pd.merge(sorted_speed,start_time,how='outer',on='auction')

In [7]:
sorted_speed=pd.merge(sorted_speed,stop_time,how='outer',on='auction')

In [24]:
plt.hist(last_time['last_time'],bins=np.arange(0,1000,100))
plt.show()

## for some auctions there is just one bid, and the last time would be 0, and I get rid of these bids

In [8]:
sorted_speed=sorted_speed[sorted_speed['last_time']!=0]

In [9]:
sorted_speed.describe()

,Unnamed: 0,bid_id,time,speed,last_time,start_time,stop_time
count,7655238.000000,7655238.000000,7.655238e+06,7.655238e+06,7.654015e+06,7.654007e+06,7.654015e+06
mean,3828281.160547,3828281.160547,9.697974e+15,5.285889e+10,4.703641e+13,9.675998e+15,9.723034e+15
std,2210152.945871,2210152.945871,5.250459e+13,1.243151e+12,3.158608e+13,5.807507e+13,3.580498e+13
min,0.000000,0.000000,9.631917e+15,5.263158e+07,3.684211e+08,9.631920e+15,9.631920e+15
25%,1914245.250000,1914245.250000,9.641139e+15,2.631579e+08,1.352180e+13,9.631920e+15,9.707650e+15
50%,3828341.500000,3828341.500000,9.700653e+15,1.315789e+09,6.462730e+13,9.631960e+15,9.709220e+15
75%,5742311.750000,5742311.750000,9.761743e+15,8.421053e+09,7.727330e+13,9.759240e+15,9.770190e+15
max,7656333.000000,7656333.000000,9.772885e+15,7.581879e+13,7.730520e+13,9.772780e+15,9.772890e+15


## here I'm trying to see if one bid time equal the end time of the auction, which means if the bidder won the auction, since it cannot help us decide if the bidder is a robot or a human

In [81]:
sorted_speed['time']==sorted_speed['stop_time']

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
...
7656335    False
7656336    False
7656337    False
7656338    False
7656339    False
7656340    False
7656341    False
7656342    False
7656343    False
7656344    False
7656345    False
7656346    False
7656347    False
7656348    False
7656349    False
Length: 7655254, dtype: bool

## here I'm trying to see at what percentage of the auction duration did the bidder bid. Does the bidder intend to bid at the beginning and then quit bidding? or does the bidder intend to start bidding when the auction is close to finish.

In [10]:
sorted_speed['bid_time_percent']=(sorted_speed['time']-sorted_speed['start_time'])/sorted_speed['last_time']

In [11]:
sorted_speed.head()

,Unnamed: 0,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url,speed,last_time,start_time,stop_time,bid_time_percent
0,5826422,5826422,92af1e40713e077ef87f5352fb56772fjnzm7,00270,mobile,phone563,9.699050e+15,us,78.188.245.105,vuaaeraf3dqz299,5.285889e+10,1.016300e+13,9.699050e+15,9.709210e+15,-0.000010
1,5834284,5834284,a939f51234ad2c44eb9ccc84e754f41foiv7g,00270,mobile,phone63,9.699098e+15,us,12.142.135.122,fbpq1mgtpgnkqnk,4.831579e+10,1.016300e+13,9.699050e+15,9.709210e+15,0.004744
2,5850448,5850448,92af1e40713e077ef87f5352fb56772fjnzm7,00270,mobile,phone735,9.699196e+15,us,2.86.254.154,nn2g552xadgkfch,9.757895e+10,1.016300e+13,9.699050e+15,9.709210e+15,0.014345
3,5884054,5884054,9655ccc7c0c193f1549475f02c54dce45kjw7,00270,sporting goods,phone573,9.699392e+15,us,149.152.163.145,vasstdc27m7nks3,1.966842e+11,1.016300e+13,9.699050e+15,9.709210e+15,0.033698
4,5898318,5898318,9655ccc7c0c193f1549475f02c54dce45kjw7,00270,sporting goods,phone2294,9.699474e+15,us,219.254.45.139,vasstdc27m7nks3,8.200000e+10,1.016300e+13,9.699050e+15,9.709210e+15,0.041767


## here I define a funciton to find the total number of bids, auctions, uniq devices, country and ip and url aggregated over all auctions

In [4]:
def extractFeatures(group):
    bid_num = len(set(group['bid_id']))
    auc_num = len(set(group['auction']))
    uniq_device = len(set(group['device']))
    uniq_country = len(set(group['country']))
    uniq_ip = len(set(group['ip']))
    uniq_url = len(set(group['url']))
    

    return pd.Series([bid_num, auc_num, uniq_device, uniq_country, uniq_ip, uniq_url])

features = grouped.apply(extractFeatures)

In [5]:
features.columns = ["bid_num", "auc_num", "uniq_device", "uniq_country", "uniq_ip", "uniq_url"]

In [17]:
grouped = sorted_speed.groupby('bidder_id')

## here first I aggregate over each auction, find the bids, country, device, ip and url for each bidder. And for each bidder, I use the maximum value out of all the auctions. The idea behind this is robots might by "hybrids". if a user shows both robots and human operated bids behavior, we should still label the user as a robot 

In [6]:
features['max_auc_bid']=sorted_speed.groupby(['bidder_id','auction'])['bid_id'].count().groupby(level=0).max()
features['max_auc_country']=sorted_speed.groupby(['bidder_id','auction'])['country'].apply(lambda x: len(x.unique())).groupby(level=0).max()
features['max_auc_device']=sorted_speed.groupby(['bidder_id','auction'])['device'].apply(lambda x: len(x.unique())).groupby(level=0).max()
features['max_auc_ip']=sorted_speed.groupby(['bidder_id','auction'])['ip'].apply(lambda x: len(x.unique())).groupby(level=0).max()
features['max_auc_url']=sorted_speed.groupby(['bidder_id','auction'])['url'].apply(lambda x: len(x.unique())).groupby(level=0).max()

## here first I aggregate over each auction, find the bids, country, device, ip and url for each bidder. And for each bidder, I calculate the mean and variance of these features over different auctions. 

In [30]:
features['mean_auc_bid']=sorted_speed.groupby(['bidder_id','auction'])['bid_id'].count().groupby(level=0).mean()
features['std_auc_bid']=sorted_speed.groupby(['bidder_id','auction'])['bid_id'].count().groupby(level=0).std()
features['mean_auc_country']=sorted_speed.groupby(['bidder_id','auction'])['country'].apply(lambda x: len(x.unique())).groupby(level=0).mean()
features['std_auc_country']=sorted_speed.groupby(['bidder_id','auction'])['country'].apply(lambda x: len(x.unique())).groupby(level=0).std()
features['mean_auc_device']=sorted_speed.groupby(['bidder_id','auction'])['device'].apply(lambda x: len(x.unique())).groupby(level=0).mean()
features['std_auc_device']=sorted_speed.groupby(['bidder_id','auction'])['device'].apply(lambda x: len(x.unique())).groupby(level=0).std()
features['mean_auc_ip']=sorted_speed.groupby(['bidder_id','auction'])['ip'].apply(lambda x: len(x.unique())).groupby(level=0).mean()
features['std_auc_ip']=sorted_speed.groupby(['bidder_id','auction'])['ip'].apply(lambda x: len(x.unique())).groupby(level=0).std()
features['mean_auc_url']=sorted_speed.groupby(['bidder_id','auction'])['url'].apply(lambda x: len(x.unique())).groupby(level=0).mean()
features['std_auc_url']=sorted_speed.groupby(['bidder_id','auction'])['url'].apply(lambda x: len(x.unique())).groupby(level=0).std()


## here I calculate the mean and variance of bidding time intervals; 
##mean and variance of bidding time percentage in each auction; 
##The winning rate of each bidder; 
##if one bidder is bidding on different auctions at exactly the same time; 
##how many consecutive bidding a bidder have in one auction; 
##mean and variance of a bidder starting and stopping time percentage in each auction

In [ ]:
features['mean_speed']=grouped.mean()['speed']
features['std_speed']=grouped.std()['speed']
features['mean_bid_time_percent']=sorted_speed.groupby(['bidder_id','auction'])['bid_time_percent'].mean().groupby(level=0).mean()
features['std_bid_time_percent']=sorted_speed.groupby(['bidder_id','auction'])['bid_time_percent'].mean().groupby(level=0).std()
features['winning_rate']=(sorted_speed.groupby(['bidder_id','auction'])['time'].last()==sorted_speed.groupby(['bidder_id','auction'])['stop_time'].last()).groupby(level=0).mean()
features['parallel_bid']=sorted_speed.groupby(['bidder_id','time'])['bid_id'].size().groupby(level=0).max()
features['consecutive_bid']=a.groupby(['auction','bidder_id'])['consec_bid'].max().groupby(level=1).mean()
features['mean_enter_time_percent']=((sorted_speed.groupby(['auction','bidder_id'])['time'].first()-sorted_speed.groupby(['auction','bidder_id'])['start_time'].first())/sorted_speed.groupby(['auction','bidder_id'])['last_time'].first()).groupby(level=1).mean()
features['std_enter_time_percent']=((sorted_speed.groupby(['auction','bidder_id'])['time'].first()-sorted_speed.groupby(['auction','bidder_id'])['start_time'].first())/sorted_speed.groupby(['auction','bidder_id'])['last_time'].first()).groupby(level=1).std()
features['mean_exit_time_percent']=((sorted_speed.groupby(['auction','bidder_id'])['time'].last()-sorted_speed.groupby(['auction','bidder_id'])['start_time'].last())/sorted_speed.groupby(['auction','bidder_id'])['last_time'].last()).groupby(level=1).mean()
features['std_exit_time_percent']=((sorted_speed.groupby(['auction','bidder_id'])['time'].last()-sorted_speed.groupby(['auction','bidder_id'])['start_time'].last())/sorted_speed.groupby(['auction','bidder_id'])['last_time'].last()).groupby(level=1).std()

In [ ]:
sorted_speed.drop(['last_time','start_time'], axis=1, inplace=True)

In [7]:
features.describe()

,bid_num,auc_num,uniq_device,uniq_country,uniq_ip,uniq_url,max_auc_bid,max_auc_country,max_auc_device,max_auc_ip,max_auc_url
count,6614.000000,6614.000000,6614.000000,6614.000000,6614.000000,6614.000000,6614.000000,6614.000000,6614.000000,6614.000000,6614.000000
mean,1157.595101,57.807832,73.437557,12.812217,544.096311,290.744935,214.599486,4.609918,25.934230,123.594648,55.870426
std,9593.019312,131.773477,172.117540,22.682700,3369.489261,2225.085049,1508.417980,10.313281,72.548414,934.328350,309.411735
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,3.000000,2.000000,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,18.000000,10.000000,8.000000,3.000000,12.000000,5.000000,4.000000,1.000000,3.000000,3.000000,2.000000
75%,187.000000,47.000000,57.000000,12.000000,110.750000,36.000000,31.000000,3.000000,14.000000,21.000000,9.000000
max,515033.000000,1726.000000,2618.000000,179.000000,111918.000000,81376.000000,62485.000000,168.000000,1273.000000,48923.000000,7497.000000


In [38]:
features['median_auc_bid']=sorted_speed.groupby(['bidder_id','auction'])['bid_id'].count().groupby(level=0).median()
features['median_auc_country']=sorted_speed.groupby(['bidder_id','auction'])['country'].apply(lambda x: len(x.unique())).groupby(level=0).median()
features['median_auc_device']=sorted_speed.groupby(['bidder_id','auction'])['device'].apply(lambda x: len(x.unique())).groupby(level=0).median()
features['median_auc_ip']=sorted_speed.groupby(['bidder_id','auction'])['ip'].apply(lambda x: len(x.unique())).groupby(level=0).median()
features['median_auc_url']=sorted_speed.groupby(['bidder_id','auction'])['url'].apply(lambda x: len(x.unique())).groupby(level=0).median()


# here I calculate for each bidder, the ratios between features. The idea is maybe human bidders can bid as much as a robot, but they ratio between some features might be different from a robot. And most linear combinations of features doesn't include this important feature.

In [39]:
features['bids_over_auc']=features['bid_num']/features['auc_num']
features['bids_over_device']=features['bid_num']/features['uniq_device']
features['bids_over_country']=features['bid_num']/features['uniq_country']
features['bids_over_ip']=features['bid_num']/features['uniq_ip']
features['bids_over_url']=features['bid_num']/features['uniq_url']
features['auc_over_device']=features['auc_num']/features['uniq_device']
features['auc_over_country']=features['auc_num']/features['uniq_country']
features['auc_over_ip']=features['auc_num']/features['uniq_ip']
features['auc_over_url']=features['auc_num']/features['uniq_url']
features['device_over_ip']=features['uniq_device']/features['uniq_ip']
features['country_over_ip']=features['uniq_country']/features['uniq_ip']

In [40]:
features.head()

,bid_num,auc_num,uniq_device,uniq_country,uniq_ip,uniq_url,mean_auc_bid,std_auc_bid,mean_auc_country,std_auc_country,...,bids_over_device,bids_over_country,bids_over_ip,bids_over_url,auc_over_device,auc_over_country,auc_over_ip,auc_over_url,device_over_ip,country_over_ip
bidder_id,,,,,,,,,,,,,,,,,,,,,
001068c415025a009fee375a12cff4fcnht8y,1,1,1,1,1,1,1.000000,NaN,1,NaN,...,1.0,1,1,1.000000,1.000,1,1.0,1.000000,1.0,1.000000
002d229ffb247009810828f648afc2ef593rb,2,1,2,1,1,1,2.000000,NaN,1,NaN,...,1.0,2,2,2.000000,0.500,1,1.0,1.000000,2.0,1.000000
0030a2dd87ad2733e0873062e4f83954mkj86,1,1,1,1,1,1,1.000000,NaN,1,NaN,...,1.0,1,1,1.000000,1.000,1,1.0,1.000000,1.0,1.000000
003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,3,3,3,1,3,2,1.000000,0.000000,1,0,...,1.0,3,1,1.500000,1.000,3,1.0,1.500000,1.0,0.333333
00486a11dff552c4bd7696265724ff81yeo9v,20,13,8,1,10,7,1.538462,0.660225,1,0,...,2.5,20,2,2.857143,1.625,13,1.3,1.857143,0.8,0.100000


## fill NAN as 0, log features and make the distribution more normalized, and scale all features and save

In [8]:
features.fillna(0).to_csv('features_0609.csv')

In [48]:
features_log=np.log(features.fillna(0)+5)

In [49]:
features_norm=features_log.apply(lambda x: StandardScaler().fit_transform(x))

In [50]:
features_norm.describe()

,bid_num,auc_num,uniq_device,uniq_country,uniq_ip,uniq_url,mean_auc_bid,std_auc_bid,mean_auc_country,std_auc_country,...,bids_over_device,bids_over_country,bids_over_ip,bids_over_url,auc_over_device,auc_over_country,auc_over_ip,auc_over_url,device_over_ip,country_over_ip
count,6.613000e+03,6.613000e+03,6.613000e+03,6.613000e+03,6.613000e+03,6.613000e+03,6.613000e+03,6.613000e+03,6.613000e+03,6.613000e+03,...,6.613000e+03,6.613000e+03,6.613000e+03,6.613000e+03,6.613000e+03,6.613000e+03,6.613000e+03,6.613000e+03,6.613000e+03,6.613000e+03
mean,-1.128187e-17,2.105949e-16,3.019243e-16,1.332335e-16,-1.826588e-17,2.578712e-16,-1.514994e-16,4.335460e-16,1.069091e-15,1.127112e-15,...,3.972829e-16,1.668105e-16,-5.802103e-17,2.216081e-16,8.756878e-16,9.149057e-16,1.046527e-15,1.402175e-16,-8.638687e-16,-8.896558e-16
std,1.000076e+00,1.000076e+00,1.000076e+00,1.000076e+00,1.000076e+00,1.000076e+00,1.000076e+00,1.000076e+00,1.000076e+00,1.000076e+00,...,1.000076e+00,1.000076e+00,1.000076e+00,1.000076e+00,1.000076e+00,1.000076e+00,1.000076e+00,1.000076e+00,1.000076e+00,1.000076e+00
min,-9.739098e-01,-1.005474e+00,-9.523685e-01,-8.147391e-01,-9.203023e-01,-7.587409e-01,-6.096045e-01,-6.504507e-01,-3.741566e-01,-4.992031e-01,...,-5.194846e-01,-8.205465e-01,-2.698880e-01,-5.629978e-01,-1.063600e+00,-1.267644e+00,-9.786343e-01,-8.665163e-01,-1.564594e+00,-1.174188e+00
25%,-8.405987e-01,-8.866629e-01,-8.437529e-01,-8.147391e-01,-8.415889e-01,-7.587409e-01,-6.096045e-01,-6.504507e-01,-3.741566e-01,-4.992031e-01,...,-5.194846e-01,-7.256254e-01,-2.698880e-01,-5.391272e-01,-3.540643e-01,-8.394938e-01,-5.135709e-01,-4.480828e-01,-5.305014e-01,-9.046054e-01
50%,-3.512268e-01,-2.992480e-01,-4.075738e-01,-4.582305e-01,-3.885084e-01,-4.546649e-01,-4.210885e-01,-4.676542e-01,-3.741566e-01,-4.992031e-01,...,-3.414690e-01,-3.908775e-01,-2.107932e-01,-3.689327e-01,-1.722840e-01,-2.894194e-01,-1.413955e-02,-3.636207e-01,8.398843e-02,-3.070695e-01
75%,6.321026e-01,6.589359e-01,6.931522e-01,4.758775e-01,5.876658e-01,3.852448e-01,1.348796e-01,2.222116e-01,-5.924069e-02,6.129041e-02,...,1.089534e-02,3.965517e-01,-9.724984e-02,1.041436e-02,-2.071247e-02,4.557576e-01,2.064260e-01,6.288908e-02,5.551462e-01,1.123559e+00
max,4.290390e+00,3.358327e+00,3.331862e+00,3.427415e+00,4.101068e+00,4.905278e+00,8.939977e+00,6.618866e+00,2.372526e+01,1.034100e+01,...,9.978380e+00,6.450622e+00,1.471111e+01,8.181350e+00,1.276789e+01,8.073291e+00,1.485347e+01,8.789371e+00,2.754038e+01,6.409590e+00


In [51]:
features_norm.to_csv('features_norm_0608.csv')